# Stock Owl

在台灣投資美股有個最惱人的地方，就是美股開盤都在半夜。如果要時時守著盤勢，對一般人來說還是太吃不消了。為了能夠協助隨時追蹤並且在發生緊急事件時能夠告警，我們列出需要完成的部分：
1. Telegram 機器人
2. Firstrade API 介接

## Telegram 機器人

In [1]:
import telegram
from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

In [2]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

In [3]:
CHOOSING, COMFIRM_STOCK, TYPING_CHOICE = range(3)

reply_keyboard = [['輸入追蹤股票', '查詢技術線圖'],
                  ['結束']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)

In [4]:
def start(update, context):
    update.message.reply_text(chat_id=update.effective_chat.id, 
                             text='''
                             歡迎使用股市貓頭鷹🦉，請選擇你想要執行的服務，我會盡全力協助你的！
                             ''',
                             reply_markup=markup)
    print(update.effective_chat.id)
    return CHOOSING

In [5]:
def input_stock(update, context):
    update.message.reply_text(chat_id=update.effective_chat.id, text='''
    請輸入你要追蹤股票代碼：
    （回上一頁請輸入：q）
    ''')
    return TYPING_CHOICE

In [1]:
def catch_stock_info(stock_id):
    import requests
    exchanges = ['NASDAQ', 'NYSE']
    apikey = 'x'
    stock_list = []

    for exchange in exchanges:
        res = requests.get('https://financialmodelingprep.com/api/v3/search?query={query}&limit=10&exchange={exchange}&apikey={apikey}'\
                           .format(query = stock_id, exchange = exchange, apikey = apikey))
        stock_list += res.json()
    return stock_list

In [7]:
def check_and_store(update, context):
    text = update.message.text
    stock_list = catch_stock_info(text)
    
    if not stock_list:
        update.message.reply_text(chat_id=update.effective_chat.id, text='''
        查詢不到你輸入的股票代碼哦，要不要檢查一下該股票代碼是否有在美股市場上市呢？再重新輸入一次吧！
        （回上一頁請輸入：q）
        ''')
        return TYPING_CHOICE
    
    stocklist_keyboard = [['\r'.join(['{}:{}'.format(k, v)]) for k, v in stock.items()] for stock in stock_list]
    stocklist_markup = ReplyKeyboardMarkup(stocklist_keyboard, one_time_keyboard=True)
    update.message.reply_text(chat_id=update.effective_chat.id,
                              text='''
                              你查詢的結果如下，請問你想加入的是第幾支標的呢？
                              ''',
                              reply_markup=stocklist_markup
                             )
    return COMFIRM_STOCK

In [8]:
def stock_added(update, context):
    text = update.message.text
    context.user_data['stock_list'].append(text)
    
    update.message.reply_text(chat_id=update.effective_chat.id, 
                             text='''
                             歡迎使用股市貓頭鷹🦉，請選擇你想要執行的服務，我會盡全力協助你的！
                             ''',
                             reply_markup=markup)
    print(context.user_data['stock_list'])
    return CHOOSING

In [9]:
def done(update, context):
    user_data = context.user_data
    if 'stock_list' in user_data:
        del user_data['stock_list']

    update.message.reply_text("再見囉！希望有幫助到你🦉")

    user_data.clear()
    return ConversationHandler.END

In [10]:
def get_k_plot(update, context):
    update.message.reply_text("還沒建置完哦，抱歉！請選擇其他功能吧！",
                             reply_markup=markup)
    return CHOOSING

In [11]:
def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("x", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^輸入追蹤股票$') & ~(Filters.command | Filters.regex('^結束$')),
                                      input_stock),
                       MessageHandler(Filters.regex('^查詢技術圖$'),
                                      get_k_plot)
                       ],

            TYPING_CHOICE: [
                MessageHandler(Filters.text & ~(Filters.command | Filters.regex('^q$')),
                               check_and_store),
                MessageHandler(Filters.regex('^q$') & ~(Filters.command),
                               start)],

            COMFIRM_STOCK: [
                MessageHandler(Filters.text & ~(Filters.command | Filters.regex('^q$')),
                               stock_added)],
        },

        fallbacks=[MessageHandler(Filters.regex('^結束$'), done)]
    )

    dp.add_handler(conv_handler)
    
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [12]:
if __name__ == '__main__':
    main()

2020-09-02 16:58:57,063 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\dispatcher.py", line 340, in process_update
    handler.handle_update(update, self, check, context)
  File "D:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\conversationhandler.py", line 465, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "D:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\handler.py", line 119, in handle_update
    return self.callback(update, context)
  File "<ipython-input-4-6337f1548b81>", line 6, in start
    reply_markup=markup)
  File "D:\ProgramData\Anaconda3\lib\site-packages\telegram\message.py", line 515, in reply_text
    return self.bot.send_message(self.chat_id, *args, **kwargs)
TypeError: send_message() got multiple values for argument 'chat_id'
2020-09-02 17:16:10,237 - telegr

In [67]:
updater.stop()